In [3]:
import numpy as np, pandas as pd
import math, random
import matplotlib, seaborn

In [49]:
train_df = pd.read_csv('./train.csv')
train_df

,profile pic,nums/length username,fullname words,nums/length fullname,name==username,description length,external URL,private,#posts,#followers,#follows,fake
0,1,0.27,0,0.00,0,53,0,0,32,1000,955,0
1,1,0.00,2,0.00,0,44,0,0,286,2740,533,0
2,1,0.10,2,0.00,0,0,0,1,13,159,98,0
3,1,0.00,1,0.00,0,82,0,0,679,414,651,0
4,1,0.00,2,0.00,0,0,0,1,6,151,126,0
...,...,...,...,...,...,...,...,...,...,...,...,...
571,1,0.55,1,0.44,0,0,0,0,33,166,596,1
572,1,0.38,1,0.33,0,21,0,0,44,66,75,1
573,1,0.57,2,0.00,0,0,0,0,4,96,339,1
574,1,0.57,1,0.00,0,11,0,0,0,57,73,1


In [ ]:
for label in train_df.keys():
    print(len(train_df[label]))

# Show data

In [ ]:
#
# Do it later :(
#

# Functions

In [103]:
class node:
    def __init__(self, examples, attribute):
        self.examples = examples
        self.attribute = attribute
        self.val = None
        self.sub = []

    def addVal(self, val):
        self.val = val

    def connect(self, subTree):
        self.sub.append(subTree)


In [17]:
def plurality_value():
    print('pValue')

In [157]:
def partition(attribute, value, label, examples, type):

    dict_ret = {
        'df_splitA':[0,0],
        'df_splitB':[0,0]
    }

    if type == "continous":
        df_splitA = examples.loc[examples[attribute] <= value]
        df_splitB = examples.loc[examples[attribute] > value]

    elif type == "discrete":
        df_splitA = examples.loc[examples[attribute] == value]
        df_splitB = examples.loc[examples[attribute] != value]


    # print(df_splitA)
    # print('===========')
    # print(df_splitB)

    dict_ret['df_splitA'][0] = len(df_splitA.loc[df_splitA[label] == 'Yes'])
    dict_ret['df_splitA'][1] = len(df_splitA.loc[df_splitA[label] == 'No'])
    dict_ret['df_splitB'][0] = len(df_splitB.loc[df_splitB[label] == 'Yes'])
    dict_ret['df_splitB'][1] = len(df_splitB.loc[df_splitB[label] == 'No'])

    return dict_ret

In [158]:
def entropy(li_2_ratio):

    li_entropy = list()
    li_rat = list()

    for li_ratio in li_2_ratio:
        
        li_rat.append(sum(li_ratio))
        
        if li_ratio[0] == 0 or li_ratio[1] == 0:
            li_entropy.append(0)
        else:
            entropy = -1
            summation = 0

            for each in li_ratio:
                summation += (each/sum(li_ratio)) * math.log2(each/sum(li_ratio))
            entropy *= summation
            li_entropy.append(round(entropy,2))
    
    return li_entropy, li_rat

def gain(li_entropy, li_rat):
    
    total = sum(li_rat)
    mulSum = 0
    for entropy, ratio in zip(li_entropy, li_rat):
        mulSum += entropy*(ratio/total)
        
    return round(1-mulSum, 2)

def get_gain(attribute, a_val, label, examples, type):
    dict_ret = partition(attribute, a_val, label, examples, type)
    # print(dict_ret.values())

    li_entropy, li_ratio = entropy(li_2_ratio=dict_ret.values())
    # print(li_entropy)
    # print(li_ratio)

    Gain = gain(li_entropy, li_ratio)
    # print(Gain)
    return Gain

In [47]:
# # p.704 example Gain(Patrons) = 0.54
# li_2 = [
#     [4,0],
#     [2,4],
#     [0,2]
# ]

# li_entropy, li_rat = entropy(li_2_ratio=li_2)
# gain(li_entropy, li_rat)

0.54

In [179]:
# Focus on ONE attribute one time
def importance(attribute, examples):

    sorted_examples = examples.sort_values(by=attribute, ascending=True)

    total_amt = len(examples)
    cur_amt = 0
    li_cur_idx = list(sorted_examples.index)[:total_amt-1]
    li_nxt_idx = list(sorted_examples.index)[1:]
    
    stop_value = 0
    li_gain_val = dict()

    for cur_idx, nxt_idx in zip(li_cur_idx, li_nxt_idx):
        cur_amt += 1
            
        if examples.loc[cur_idx][attribute] != examples.loc[nxt_idx][attribute]:
            # print(cur_amt)
            # print(examples.loc[cur_idx])
            # print(examples.loc[nxt_idx])
            # print('===================')
            stop_value = examples.loc[cur_idx][attribute]
            gain = get_gain(attribute, stop_value, 'Label', examples, 'discrete')
            li_gain_val[gain] = stop_value

    
    stop_value = examples.loc[li_nxt_idx[-1]][attribute]
    gain = get_gain(attribute, stop_value, 'Label', examples, 'discrete')
    li_gain_val[gain] = stop_value
    
    print(sorted(li_gain_val.items(), key = lambda s: s[0], reverse=True))
    



In [99]:
def same_classification(examples):

    label = examples['fake'][0]

    for fake in examples['fake']:
        if fake != label:
            return False

    return True

In [32]:
# dic = {'fake':[1,1,0,1]}
# df_dic = pd.DataFrame(dic)

# print(same_classification(df_dic))

False


In [47]:
def select_attribute(dic):

    maxVal = 0
    iniKey = list(dic.keys())[0]


    maxValLabel = dic[iniKey]
    for key in dic.keys():
        if dic[key] > maxVal:
            maxValLabel = key
        
    return maxValLabel

In [50]:
# dic = {'a':0.43, 'b':0.12, 'c':0.65}

# print(select_attribute(dic))

c


In [72]:
def decision_tree_learning(examples, attributes, parent_examples):
    
    if len(examples) == 0:
        return plurality_value(parent_examples)
    
    elif(same_classification(examples)):
        return examples['fake'][0]
    
    elif(len(attributes) == 0):
        return plurality_value(examples)
    
    else:
        attribute = select_attribute(importance(attributes, examples))
        # tree assignment with attribute A
        tree = node(examples, attribute)
        # for
        #     exs = 
        #     subTree = decision_tree_learning(exs, attribute-A, examples)
        #     tree.connect(subTree)

        return tree
        
        

        


## Test

In [119]:
test_df = pd.DataFrame({
    '#follows': [1,2,3,4,5,6,7,8,9,10,11,12], 
    'Patrons': ['Some', 'Full', 'Some', 'Full', 'Full', 'Some', 'None', 'Some', 'Full', 'Full', 'None', 'Full'],
    'Type': ['French', 'Thai', 'Burger', 'Thai', 'French', 'Italian', 'Burger', 'Thai', 'Burger', 'Italian', 'Thai', 'Burger'],
    'Label': ['Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes']
})
test_df

,#follows,Patrons,Type,Label
0,1,Some,French,Yes
1,2,Full,Thai,No
2,3,Some,Burger,Yes
3,4,Full,Thai,Yes
4,5,Full,French,No
5,6,Some,Italian,Yes
6,7,None,Burger,No
7,8,Some,Thai,Yes
8,9,Full,Burger,No
9,10,Full,Italian,No


In [162]:
test_df.sort_values(by='Patrons')

,#follows,Patrons,Type,Label
1,2,Full,Thai,No
3,4,Full,Thai,Yes
4,5,Full,French,No
8,9,Full,Burger,No
9,10,Full,Italian,No
11,12,Full,Burger,Yes
6,7,None,Burger,No
10,11,None,Thai,No
0,1,Some,French,Yes
2,3,Some,Burger,Yes


In [180]:

dict_

li_gain_ret = importance('Patrons', test_df)


    

[(0.46, 'Some'), (0.19, 'None'), (0.08, 'Full')]


# Build a Decision Tree

In [96]:
root = node(examples=train_df, )